# Sediment production, transport and deposition in both continental and marine environments

This notebook shows an run example of the `marine_model` provided in [fastscape](https://fastscape.readthedocs.io/en/latest/)

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import and inspect the model

In [ ]:
from fastscape.models import marine_model

In [ ]:
marine_model

In [ ]:
marine_model.visualize(show_inputs=True)

## Model setup

In [ ]:
ds_in = xs.create_setup(
    model=marine_model,
    clocks={
        'time': np.arange(0, 3e6 + 1e3, 1e3),
        'out': np.arange(0, 3e6 + 2e5, 2e5),
        'step4': [8e3]
    },
    master_clock='time',
    input_vars={
        'grid__shape': ('shape_yx', [101, 101]),
        'grid__length': ('shape_yx', [4e5, 4e5]),
        'boundary__status': ('border', ['fixed_value', 'whatever', 'looped', 'looped']),
        'init_topography': {
            'x_left': 0,
            'x_right': 2.5e5,
            'elevation_left': -1e3,
            'elevation_right': 0.
        },
        'uplift': {
            'x_position': 2.5e5,
            'rate_left': 0.,
            'rate_right': 3e-4
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-5,
            'k_coef_soil': 1e-5,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
            'area_exp': 0.4,
            'slope_exp': 1.
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1e-2
        },
        'sea__level': 0.,
        'marine': {
            'ss_ratio_land': 0.5,
            'porosity_sand': 0.,
            'porosity_silt': 0.,
            'e_depth_sand': 1e3,
            'e_depth_silt': 1e3,
            'diffusivity_sand': 5e2,
            'diffusivity_silt': 2.5e2,
            'layer_depth': 1e2,
        },
        'strati': {
            'freeze_time': ('horizon', np.linspace(0, 3e6, 5))
        }
    },
    output_vars={
        'out': ['topography__elevation', 'strati__elevation', 'strati__active'],
        None: ['boundary__border',
               'grid__x',
               'grid__y'],
    }
)

ds_in

## Run the model

This might take 1-2 minutes...

In [ ]:
ds_out = (ds_in.xsimlab.run(model=marine_model)
               .set_index(x='grid__x', y='grid__y',
                          border='boundary__border',
                          horizon='strati__freeze_time'))

In [ ]:
ds_out

## Plot the outputs (stratigraphic horizons)

In [ ]:
import hvplot.xarray

ds_out.strati__elevation.hvplot.image(x='x', y='y', clim=(-1000, 1200),
                                      width=600, height=550,
                                      cmap=plt.cm.viridis,
                                      groupby=['horizon', 'out'])

Plot the stratigraphic horizons on a cross-section along the x-axis:

In [ ]:
ds_out.strati__elevation.sel(y=2e5).isel(out=-1).plot.line(x='x');